In [1]:
import json
import subprocess

import duckdb
import polars as pl

In [2]:
import os
import sys
import django

# Add your project root to the Python path if necessary
sys.path.append('/home/reepoi/GitHub/ProductRegistry') 

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
# Set the DJANGO_SETTINGS_MODULE environment variable
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'product_registry.settings')

# Initialize Django
django.setup()

In [3]:
from ob_taxonomy import models

In [4]:
TAXONOMY_PATH = 'Master-OB-OpenAPI.json'

In [5]:
def run_jq(query):
    return subprocess.run(
        ['jq', '-c', query, TAXONOMY_PATH],
        capture_output=True,
        text=True,
        check=True,
    ).stdout

### OB Primitive

In [6]:
ob_primitive = (
    pl.DataFrame(json.loads(
        run_jq('.components.schemas | to_entries | map(select(.key | startswith("Value") or IN("Unit", "Decimals", "Precision", "StartTime", "EndTime"))) | map({name: .key, description: .value.description, is_array: (.key | contains("Array"))})')
    ))
)
ob_primitive

name,description,is_array
str,str,bool
"""ValueNumber""","""Value for the instance element""",false
"""ValueString""","""Value for the instance element""",false
"""ValueBoolean""","""Value for the instance element""",false
"""ValueInteger""","""Value for the instance element""",false
"""ValueArrayNumber""","""Value for the instance element""",true
…,…,…
"""Unit""","""Unit of the instance element""",false
"""Decimals""","""Decimals of the instance eleme…",false
"""Precision""","""Precision of the instance elem…",false


In [7]:
models.OBPrimitive.objects.bulk_create([
    models.OBPrimitive(**row) for row in ob_primitive.iter_rows(named=True)
])

[<OBPrimitive: ValueNumber>,
 <OBPrimitive: ValueString>,
 <OBPrimitive: ValueBoolean>,
 <OBPrimitive: ValueInteger>,
 <OBPrimitive: ValueArrayNumber>,
 <OBPrimitive: ValueArrayString>,
 <OBPrimitive: ValueArrayBoolean>,
 <OBPrimitive: ValueArrayInteger>,
 <OBPrimitive: Unit>,
 <OBPrimitive: Decimals>,
 <OBPrimitive: Precision>,
 <OBPrimitive: StartTime>,
 <OBPrimitive: EndTime>]

### OB Taxonomy Element

In [8]:
ob_taxonomy_element = (
    pl.DataFrame(json.loads(
        run_jq('.components.schemas | to_entries | map(select(.key | contains("TaxonomyElement"))) | map({name: .key, primitive: .value.properties[]["$ref"]})')
    ))
    .with_columns(primitive=pl.col('primitive').str.split('/').list[-1])
)
ob_taxonomy_element

name,primitive
str,str
"""TaxonomyElementString""","""ValueString"""
"""TaxonomyElementString""","""Unit"""
"""TaxonomyElementString""","""Decimals"""
"""TaxonomyElementString""","""Precision"""
"""TaxonomyElementString""","""StartTime"""
…,…
"""TaxonomyElementArrayInteger""","""Unit"""
"""TaxonomyElementArrayInteger""","""Decimals"""
"""TaxonomyElementArrayInteger""","""Precision"""


In [9]:
ob_primitive_rows = models.OBPrimitive.objects.all()
ob_taxonomy_element_rows = models.OBTaxonomyElement.objects.bulk_create([
    models.OBTaxonomyElement(**row) for row in ob_taxonomy_element.select(pl.exclude('primitive')).unique().iter_rows(named=True)
])
for taxonomy_element in ob_taxonomy_element_rows:
    taxonomy_element.primitives.add(
        ob_primitive_rows.get(name=f'Value{taxonomy_element.name.split('TaxonomyElement')[-1]}'),
        ob_primitive_rows.get(name='StartTime'),
        ob_primitive_rows.get(name='EndTime'),
    )
    if (
        taxonomy_element.name.endswith('Integer')
        or
        taxonomy_element.name.endswith('Number')
    ):
        taxonomy_element.primitives.add(
            ob_primitive_rows.get(name='Unit'),
            ob_primitive_rows.get(name='Decimals'),
            ob_primitive_rows.get(name='Precision'),
        )

### OB Item Type Enum

In [10]:
ob_item_type_enum = (
    pl.DataFrame(json.loads(
        run_jq('.["x-ob-item-types"] | [.[].enums | select(. != null)] | add | to_entries | map({name: .key} + .value)')
    ))
)
ob_item_type_enum

name,label,description
str,str,str
"""POA""","""Plane of Array""","""Irradiance, both direct and di…"
"""GHI""","""Global Horizontal""","""Irradiance, both direct and di…"
"""DNI""","""Direct Normal""","""Irradiance from the sun direct…"
"""DHI""","""Diffuse Horizontal""","""Irradiance scattered by the at…"
"""Other""","""Other""",""""""
…,…,…
"""YE""","""Yemen""",""""""
"""YT""","""Mayotte""",""""""
"""ZA""","""South Africa""",""""""


In [11]:
models.OBItemTypeEnum.objects.bulk_create([
    models.OBItemTypeEnum(**row) for row in ob_item_type_enum.iter_rows(named=True)
])

[<OBItemTypeEnum: Plane of Array (POA)>,
 <OBItemTypeEnum: Global Horizontal (GHI)>,
 <OBItemTypeEnum: Direct Normal (DNI)>,
 <OBItemTypeEnum: Diffuse Horizontal (DHI)>,
 <OBItemTypeEnum: Other (Other)>,
 <OBItemTypeEnum: Surface (Surface)>,
 <OBItemTypeEnum: Ambient (Ambient)>,
 <OBItemTypeEnum: Internal (Internal)>,
 <OBItemTypeEnum: Insulated-Gate Bipolar Transistor (IGBT)>,
 <OBItemTypeEnum: CPU (CPU)>,
 <OBItemTypeEnum: Panel (Panel)>,
 <OBItemTypeEnum:  (Ground)>,
 <OBItemTypeEnum: Central (Central)>,
 <OBItemTypeEnum: String (String)>,
 <OBItemTypeEnum: MicroInverter (MicroInverter)>,
 <OBItemTypeEnum:  (Distributed)>,
 <OBItemTypeEnum: Transformerless (Transformerless)>,
 <OBItemTypeEnum: Grounded (Grounded)>,
 <OBItemTypeEnum: BIPV (BIPV)>,
 <OBItemTypeEnum: Bifacial PV (Bifacial)>,
 <OBItemTypeEnum: Concentrating PV (CPV)>,
 <OBItemTypeEnum:  (Ribbon)>,
 <OBItemTypeEnum: Monofacial PV (MonoPV)>,
 <OBItemTypeEnum:  (AzimuthAxisTracking)>,
 <OBItemTypeEnum:  (FixedTilt)>,
 <OBI

### OB Item Type Unit

In [12]:
ob_item_type_unit = (
    pl.DataFrame(json.loads(
        run_jq('.["x-ob-item-types"] | [.[].units | select(. != null)] | add | to_entries | map({name: .key} + .value)')
    ))
)
ob_item_type_unit

name,label,description
str,str,str
"""acre""","""Acre""","""Acre"""
"""sqft""","""Square Foot""","""Square Foot"""
"""sqmi""","""Square Mile""","""Square Miles"""
"""sqyd""","""Square Yard""","""Square Yard"""
"""ha""","""Hectare""","""Hectare"""
…,…,…
"""EUR_per_W""","""Euro per Watt""",""""""
"""EUR_per_kW""","""Euro per kilowatt""",""""""
"""EUR_per_MW""","""Euro per megawatt""",""""""


In [13]:
models.OBItemTypeUnit.objects.bulk_create([
    models.OBItemTypeUnit(**row) for row in ob_item_type_unit.iter_rows(named=True)
])

[<OBItemTypeUnit: Acre (acre)>,
 <OBItemTypeUnit: Square Foot (sqft)>,
 <OBItemTypeUnit: Square Mile (sqmi)>,
 <OBItemTypeUnit: Square Yard (sqyd)>,
 <OBItemTypeUnit: Hectare (ha)>,
 <OBItemTypeUnit: Square km (sqkm)>,
 <OBItemTypeUnit: Square metre (sqm)>,
 <OBItemTypeUnit: Barrel of Oil Equivalent (Boe)>,
 <OBItemTypeUnit: British Thermal Unit (Btu)>,
 <OBItemTypeUnit: Foot-Pound (ft_lb)>,
 <OBItemTypeUnit: Thousand Barrels of Oil Equivalent (MBoe)>,
 <OBItemTypeUnit: Thousand Cubic Foot Equivalent (Mcfe)>,
 <OBItemTypeUnit: Millions of Barrels of Oil Equivalent (MMBoe)>,
 <OBItemTypeUnit: Millions of BTU (MMBTU)>,
 <OBItemTypeUnit: Calorie (Cal)>,
 <OBItemTypeUnit: Joule (J)>,
 <OBItemTypeUnit: Kilojoule (kJ)>,
 <OBItemTypeUnit: Kilowatt-Hours (kWh)>,
 <OBItemTypeUnit: mJ (mJ)>,
 <OBItemTypeUnit: Megawatt-Hour (MWh)>,
 <OBItemTypeUnit: Gigawatt-Hour (GWh)>,
 <OBItemTypeUnit: Terawatt-Hour (TWh)>,
 <OBItemTypeUnit: Million Cubic Foot Equivalent (MMcfe)>,
 <OBItemTypeUnit: Billion Cub

### OB Item Type

In [14]:
ob_item_type = pl.concat([
    pl.DataFrame(json.loads(  # has enums
        run_jq('.["x-ob-item-types"] | to_entries | map({name: .key, description: .value.description} + (.value.enums | select(. != null) | to_entries | map({enum_name: .key, enum_label: .value.label})[]))')
    )),
    pl.DataFrame(json.loads(  # has units
        run_jq('.["x-ob-item-types"] | to_entries | map({name: .key, description: .value.description} + (.value.units | select(. != null) | to_entries | map({unit_name: .key, unit_label: .value.label})[]))')
    )),
    pl.DataFrame(json.loads(  # no enums or units
        run_jq('.["x-ob-item-types"] | to_entries | map(select(.value.enums == null and .value.units == null)) | map({name: .key} + .value)')
    )),
], how='diagonal')
ob_item_type

name,description,enum_name,enum_label,unit_name,unit_label
str,str,str,str,str,str
"""IrradianceTypeItemType""","""Defines the type of irradiance…","""POA""","""Plane of Array""",null,null
"""IrradianceTypeItemType""","""Defines the type of irradiance…","""GHI""","""Global Horizontal""",null,null
"""IrradianceTypeItemType""","""Defines the type of irradiance…","""DNI""","""Direct Normal""",null,null
"""IrradianceTypeItemType""","""Defines the type of irradiance…","""DHI""","""Diffuse Horizontal""",null,null
"""IrradianceTypeItemType""","""Defines the type of irradiance…","""Other""","""Other""",null,null
…,…,…,…,…,…
"""StringItemType""","""Enum source prefixes: stdi""",null,null,null,null
"""IntegerItemType""","""Enum source prefixes: stdi""",null,null,null,null
"""TextBlockItemType""","""textBlockItemType specializes …",null,null,null,null


In [15]:
models.OBItemType.objects.bulk_create([
    models.OBItemType(**row) for row in ob_item_type.select(pl.exclude(r'^(enum|unit)_.*$')).unique().iter_rows(named=True)
])
ob_item_type_rows = models.OBItemType.objects.all()
for item_type in ob_item_type_rows:
    enums = ob_item_type.filter(name=item_type.name).filter(pl.col('enum_name').is_null().not_())
    if len(enums) > 0:
        enums = models.OBItemTypeEnum.objects.filter(name__in=enums['enum_name'], label__in=enums['enum_label'])
        item_type.enums.set(enums)
    else:
        units = ob_item_type.filter(name=item_type.name).filter(pl.col('unit_name').is_null().not_())
        units = models.OBItemTypeUnit.objects.filter(name__in=units['unit_name'], label__in=units['unit_label'])
        item_type.units.set(units)

### OB Item Type Group

In [16]:
ob_item_type_group = (
    pl.DataFrame(json.loads(
        run_jq('.["x-ob-item-type-groups"] | to_entries | map({name: .key, item_type: .value.type, description: .value.description, enum_or_unit_name: .value.group[]})')
    ))
    .with_columns(item_type=pl.col('item_type').str.split('/').list[-1])
)
ob_item_type_group

name,item_type,description,enum_or_unit_name
str,str,str,str
"""OBMonetary""","""MonetaryItemType""","""""","""CAD"""
"""OBMonetary""","""MonetaryItemType""","""""","""EUR"""
"""OBMonetary""","""MonetaryItemType""","""""","""USD"""
"""OBElectricalEnergy""","""EnergyItemType""","""""","""kWh"""
"""OBElectricalEnergy""","""EnergyItemType""","""""","""MWh"""
…,…,…,…
"""MassUnitItemType""","""MassItemType""","""Restriction of MassItemType to…","""g"""
"""MassUnitItemType""","""MassItemType""","""Restriction of MassItemType to…","""kg"""
"""MassUnitItemType""","""MassItemType""","""Restriction of MassItemType to…","""kt"""


In [17]:
ob_item_type_rows = models.OBItemType.objects.all()
models.OBItemTypeGroup.objects.bulk_create([
    models.OBItemTypeGroup(**{**row, 'item_type': ob_item_type_rows.get(name=row['item_type'])})
    for row in ob_item_type_group.select(pl.exclude('enum_or_unit_name')).unique().iter_rows(named=True)
])
ob_item_type_group_rows = models.OBItemTypeGroup.objects.all()
for item_type_group in ob_item_type_group_rows:
    value_set = ob_item_type_group.filter(name=item_type_group.name)
    if item_type_group.item_type.enums.exists():
        enums = item_type_group.item_type.enums.filter(name__in=value_set['enum_or_unit_name'])
        item_type_group.enums.set(enums)
    else:
        units = item_type_group.item_type.units.filter(name__in=value_set['enum_or_unit_name'])
        item_type_group.units.set(units)

### OB Element

In [18]:
ob_element = (
    pl.DataFrame(json.loads(
        run_jq('.components.schemas | to_entries | map(select(.value.allOf[0]["$ref"] | (. != null and contains("TaxonomyElement")))) | map({name: .key, taxonomy_element: .value.allOf[0]["$ref"]} + (.value.allOf[1] | {description: .description, item_type: ."x-ob-item-type", item_type_group: ."x-ob-item-type-group"}))')
    ))
    .with_columns(
        taxonomy_element=pl.col('taxonomy_element').str.split('/').list[-1],
        item_type=pl.col('item_type').str.split('/').list[-1],
        item_type_group=pl.col('item_type_group').str.split('/').list[-1],
    )
)
ob_element

name,taxonomy_element,description,item_type,item_type_group
str,str,str,str,str
"""City""","""TaxonomyElementString""","""Name of the city in the physic…","""StringItemType""",""""""
"""StateProvince""","""TaxonomyElementString""","""State of a Province of the mai…","""StringItemType""",""""""
"""ZipPostalCode""","""TaxonomyElementString""","""Zip Code / Postal Code of a ma…","""StringItemType""",""""""
"""Country""","""TaxonomyElementString""","""ISO country code where system …","""StringItemType""",""""""
"""Latitude""","""TaxonomyElementNumber""","""Decimal value of latitude of t…","""PlaneAngleItemType""",""""""
…,…,…,…,…
"""ChecklistTemplateMaintainerID""","""TaxonomyElementString""","""Identifier for the maintainer …","""UUIDItemType""",""""""
"""ChecklistTemplateMaintainerNam…","""TaxonomyElementString""","""Name of the entity that mainta…","""StringItemType""",""""""
"""SectionName""","""TaxonomyElementString""","""A section creates a grouping o…","""StringItemType""",""""""


In [19]:
ob_taxonomy_element_rows = models.OBTaxonomyElement.objects.all()
ob_item_type_rows = models.OBItemType.objects.all()
ob_item_type_group_rows = models.OBItemTypeGroup.objects.all()
models.OBElement.objects.bulk_create([
    models.OBElement(**{
        **row,
        'taxonomy_element': ob_taxonomy_element_rows.get(name=row['taxonomy_element']),
        'item_type': ob_item_type_rows.get(name=row['item_type']),
        'item_type_group': ob_item_type_group_rows.get(name=row['item_type_group']) if row['item_type_group'] != '' else None,
    })
    for row in ob_element.iter_rows(named=True)
])

[<OBElement: City>,
 <OBElement: StateProvince>,
 <OBElement: ZipPostalCode>,
 <OBElement: Country>,
 <OBElement: Latitude>,
 <OBElement: Longitude>,
 <OBElement: County>,
 <OBElement: Email>,
 <OBElement: MobilePhone>,
 <OBElement: WorkPhone>,
 <OBElement: HomePhone>,
 <OBElement: FirstName>,
 <OBElement: MiddleName>,
 <OBElement: LastName>,
 <OBElement: Altitude>,
 <OBElement: Elevation>,
 <OBElement: AddrLine1>,
 <OBElement: AddrLine2>,
 <OBElement: AddrLine3>,
 <OBElement: EnergyModelName>,
 <OBElement: EnergyModelConfigurationFilename>,
 <OBElement: Description>,
 <OBElement: BuildingCode>,
 <OBElement: BuildingCodeNotes>,
 <OBElement: ResidentialCode>,
 <OBElement: FireCode>,
 <OBElement: ElectricCode>,
 <OBElement: ResidentialCodeNotes>,
 <OBElement: FireCodeNotes>,
 <OBElement: ElectricCodeNotes>,
 <OBElement: DocumentSubmissionMethod>,
 <OBElement: DocumentSubmissionMethodNotes>,
 <OBElement: EngineeringReviewType>,
 <OBElement: StampType>,
 <OBElement: AHJName>,
 <OBElement: 

### OB Object

In [20]:
ob_object_properties = (
    pl.DataFrame(
        json.loads(
            run_jq('.components.schemas | to_entries | map(select(.value.type == "object")) | map(select(.key | (contains("TaxonomyElement")|not))) | map({name: .key, description: (.value.description // ""), property: (.value.properties[]["$ref"] // null)})')
        )
        +
        json.loads(
            run_jq('.components.schemas | to_entries | map(select(.value.allOf[0]["$ref"] | (. != null and (contains("TaxonomyElement")|not)))) | map(.value = .value.allOf[1]) | map({name: .key, description: .value.description, property: (.value.properties[]["$ref"] // null)})')
            # run_jq('.components.schemas | to_entries | map(select(.value.allOf[0]["$ref"] | (. != null and (contains("TaxonomyElement")|not)))) | map(.value = .value.allOf[1]) | map({name: .key, description: .value.description})')
        )
    )
    .with_columns(property=pl.col('property').str.split('/').list[-1])
)
ob_object_properties

name,description,property
str,str,str
"""EnergyModel""","""Energy model contains energy p…","""EnergyModelName"""
"""EnergyModel""","""Energy model contains energy p…","""Description"""
"""EnergyModel""","""Energy model contains energy p…","""EnergyModelInput"""
"""EnergyModel""","""Energy model contains energy p…","""EnergyModelOutput"""
"""EnergyModel""","""Energy model contains energy p…","""ModelRunDate"""
…,…,…
"""QualifyingAgency""","""Agency that lists the Product …","""QualifyingAgencyID"""
"""QualifyingAgency""","""Agency that lists the Product …","""QualifyingAgencyName"""
"""IssueRecommendation""","""Recommendation to respond to a…",null


In [21]:
ob_object_properties.filter(name='Question', property='AnswerOptions')

name,description,property
str,str,str
"""Question""","""A question for an Checklist""","""AnswerOptions"""


In [22]:
models.OBObject.objects.bulk_create([
    models.OBObject(**row)
    for row in ob_object_properties.select(pl.exclude('property')).unique().iter_rows(named=True)
])
ob_object_rows = models.OBObject.objects.all()
for _object in ob_object_rows:
    elements = ob_object_properties.filter(name=_object.name)
    elements = models.OBElement.objects.filter(name__in=elements['property'])
    _object.properties.set(elements)

### OB Array

In [23]:
ob_array = (
    pl.DataFrame(json.loads(
        run_jq('.components.schemas | to_entries | map(select(.value.type == "array")) | map({name: .key, items: .value.items["$ref"]})')
    ))
    .with_columns(items=pl.col('items').str.split('/').list[-1])
    .with_columns(item_is_element=pl.col('items').is_in(ob_element['name'].implode()))
)
ob_array

name,items,item_is_element
str,str,bool
"""EngineeringReviewRequirements""","""EngineeringReviewRequirement""",false
"""Addresses""","""Address""",false
"""Contacts""","""Contact""",false
"""Tasks""","""Task""",false
"""Devices""","""Device""",false
…,…,…
"""CurrentArray""","""Current""",true
"""PowerFactorArray""","""PowerFactor""",true
"""LineFrequencies""","""LineFrequency""",true


In [24]:
ob_element_rows = models.OBElement.objects.all()
ob_object_rows = models.OBObject.objects.all()
models.OBArrayOfElement.objects.bulk_create([
    models.OBArrayOfElement(name=row['name'], items=ob_element_rows.get(name=row['items']))
    for row in ob_array.filter(item_is_element=True).iter_rows(named=True)
])
models.OBArrayOfObject.objects.bulk_create([
    models.OBArrayOfObject(name=row['name'], items=ob_object_rows.get(name=row['items']))
    for row in ob_array.filter(item_is_element=False).iter_rows(named=True)
])

[<OBArrayOfObject: EngineeringReviewRequirements>,
 <OBArrayOfObject: Addresses>,
 <OBArrayOfObject: Contacts>,
 <OBArrayOfObject: Tasks>,
 <OBArrayOfObject: Devices>,
 <OBArrayOfObject: PVSystems>,
 <OBArrayOfObject: ProdLineItems>,
 <OBArrayOfObject: Structures>,
 <OBArrayOfObject: ElectricalServices>,
 <OBArrayOfObject: FeeStructures>,
 <OBArrayOfObject: Fees>,
 <OBArrayOfObject: Answers>,
 <OBArrayOfObject: Questions>,
 <OBArrayOfObject: AccountBalances>,
 <OBArrayOfObject: AgingBuckets>,
 <OBArrayOfObject: EnergyConsumptions>,
 <OBArrayOfObject: ProdSpecifications>,
 <OBArrayOfObject: ProdCertifications>,
 <OBArrayOfObject: SnowLoads>,
 <OBArrayOfObject: AHJInspections>,
 <OBArrayOfObject: BatterySystems>,
 <OBArrayOfObject: WindLoads>,
 <OBArrayOfObject: ServiceLineItems>,
 <OBArrayOfObject: WindDirectionTSArray>,
 <OBArrayOfObject: PVStrings>,
 <OBArrayOfObject: ProductLineItems>,
 <OBArrayOfObject: PVArrays>,
 <OBArrayOfObject: RoofPlanes>,
 <OBArrayOfObject: Comments>,
 <OBArr

### OB Object Relationships

##### allOf, oneOf, anyOf

In [25]:
ob_object_comprisal = (
    pl.DataFrame(json.loads(
        run_jq('.components.schemas | to_entries | map(select(.value.allOf[0]["$ref"] | (. != null and (contains("TaxonomyElement")|not)))) | map({source: .value.allOf[0]["$ref"], target: .key, method: "allOf"})')
    ))
    .with_columns(source=pl.col('source').str.split('/').list[-1])
)
ob_object_comprisal

source,target,method
str,str,str
"""Product""","""ProdCombiner""","""allOf"""
"""Product""","""ProdInverter""","""allOf"""
"""Product""","""ProdWire""","""allOf"""
"""Product""","""ProdOptimizer""","""allOf"""
"""Product""","""ProdModule""","""allOf"""
…,…,…
"""Entity""","""ServiceRequester""","""allOf"""
"""Contract""","""ContractES""","""allOf"""
"""Entity""","""QualifyingAgency""","""allOf"""


In [26]:
for target in ob_object_comprisal['target'].unique():
    comprisals = ob_object_comprisal.filter(target=target).sort('source')
    composition = models.OBObject.objects.filter(name__in=comprisals['source']).order_by('name')
    target = models.OBObject.objects.get(name=target)
    comprisals = models.OBObjectComprisal.objects.bulk_create([
        models.OBObjectComprisal(source=composed, target=target, method=method)
        for composed, method in zip(composition, comprisals['method'])
    ])

##### Nested OB Objects

In [27]:
ob_objects_and_their_nested_objects =  ob_object_properties.filter(pl.col('property').is_in(
    list(models.OBObject.objects.filter(name__in=ob_object_properties['property']).values_list('name', flat=True))
))
ob_objects_and_their_nested_objects

name,description,property
str,str,str
"""EnergyModel""","""Energy model contains energy p…","""EnergyModelInput"""
"""EnergyModel""","""Energy model contains energy p…","""EnergyModelOutput"""
"""EnergyModel""","""Energy model contains energy p…","""Scope"""
"""Product""","""An item or equipment that is p…","""Dimension"""
"""Device""","""A singular instance of a Produ…","""Firmware"""
…,…,…
"""HomeownersAssociation""","""The Homeowner's Association of…","""Address"""
"""OMTask""","""A Task object customized for O…","""TaskBudget"""
"""OMTask""","""A Task object customized for O…","""OMWorkPlan"""


In [28]:
ob_objects_with_nested_objects = models.OBObject.objects.filter(name__in=ob_objects_and_their_nested_objects['name'].unique())
for ob_object in ob_objects_with_nested_objects:
    ob_object.nested_objects.set(models.OBObject.objects.filter(name__in=ob_objects_and_their_nested_objects.filter(name=ob_object.name)['property']))

##### Nested OB Arrays

In [29]:
ob_objects_and_their_nested_element_arrays =  ob_object_properties.filter(pl.col('property').is_in(
    list(models.OBArrayOfElement.objects.filter(name__in=ob_object_properties['property']).values_list('name', flat=True))
)).with_columns(item_is_element=True)
ob_objects_and_their_nested_element_arrays

name,description,property,item_is_element
str,str,str,bool
"""Product""","""An item or equipment that is p…","""ProdInstructions""",true
"""Device""","""A singular instance of a Produ…","""CryptographicKeyIDs""",true
"""EnergyConsumption""","""Energy consumed from all sourc…","""EnergyACArray""",true
"""Checklist""","""An Checklist or survey""","""Tags""",true
"""Answer""","""The result of an Checklist que…","""AnswerValues""",true
…,…,…,…
"""PowerACMeas""","""An object containing PowerAC m…","""PowerACArray""",true
"""TempAmbMeas""","""An object containing Ambient T…","""TempAmbArray""",true
"""IrradPlaneOfArrayMeas""","""Measurements of plane-of-array…","""IrradPlaneOfArrayArray""",true


In [30]:
ob_objects_and_their_nested_object_arrays =  ob_object_properties.filter(pl.col('property').is_in(
    list(models.OBArrayOfObject.objects.filter(name__in=ob_object_properties['property']).values_list('name', flat=True))
)).with_columns(item_is_element=False)
ob_objects_and_their_nested_object_arrays

name,description,property,item_is_element
str,str,str,bool
"""Product""","""An item or equipment that is p…","""ProdSpecifications""",false
"""Product""","""An item or equipment that is p…","""ProdCertifications""",false
"""Product""","""An item or equipment that is p…","""Packages""",false
"""Product""","""An item or equipment that is p…","""Warranties""",false
"""Product""","""An item or equipment that is p…","""AlternativeIdentifiers""",false
…,…,…,…
"""AuthorityHavingJurisdiction""","""The authority having jurisdict…","""EngineeringReviewRequirements""",false
"""AuthorityHavingJurisdiction""","""The authority having jurisdict…","""FeeStructures""",false
"""AuthorityHavingJurisdiction""","""The authority having jurisdict…","""AHJInspections""",false


In [31]:
ob_objects_and_their_nested_arrays = pl.concat([ob_objects_and_their_nested_element_arrays, ob_objects_and_their_nested_object_arrays])
ob_objects_and_their_nested_arrays

name,description,property,item_is_element
str,str,str,bool
"""Product""","""An item or equipment that is p…","""ProdInstructions""",true
"""Device""","""A singular instance of a Produ…","""CryptographicKeyIDs""",true
"""EnergyConsumption""","""Energy consumed from all sourc…","""EnergyACArray""",true
"""Checklist""","""An Checklist or survey""","""Tags""",true
"""Answer""","""The result of an Checklist que…","""AnswerValues""",true
…,…,…,…
"""AuthorityHavingJurisdiction""","""The authority having jurisdict…","""EngineeringReviewRequirements""",false
"""AuthorityHavingJurisdiction""","""The authority having jurisdict…","""FeeStructures""",false
"""AuthorityHavingJurisdiction""","""The authority having jurisdict…","""AHJInspections""",false


In [32]:
ob_objects_with_nested_arrays = models.OBObject.objects.filter(name__in=ob_objects_and_their_nested_arrays['name'].unique())
for ob_object in ob_objects_with_nested_arrays:
    ob_object.element_arrays.set(models.OBArrayOfElement.objects.filter(name__in=ob_objects_and_their_nested_arrays.filter(name=ob_object.name, item_is_element=True)['property']))
    ob_object.object_arrays.set(models.OBArrayOfObject.objects.filter(name__in=ob_objects_and_their_nested_arrays.filter(name=ob_object.name, item_is_element=False)['property']))